In [ ]:
import pyaudio
import queue
import numpy as np
import torch
import torchaudio
import wave

In [ ]:
def get_demo_wrapper():
    wrapper = torch.jit.load("scripted_wrapper_tuple.pt")
    return wrapper

In [ ]:
def callback(in_data, frame_count, time_info, status):
    global data_queue, wf
    # data_queue.put(in_data)
    data = wf.readframes(frame_count)
    data_queue.put(data)
    print(frame_count)
    # return in_data, pyaudio.paContinue
    return data, pyaudio.paContinue

In [ ]:
def transcribe(np_array, should_print=True):
    global state, hypo
    tensor = torch.tensor(np_array)
    transcript, hypo, state = wrapper(tensor, hypo, state)
    if should_print and transcript:
        print(transcript, end="", flush=True)

In [ ]:
def process(should_print=True):
    global previous_right_context
    if previous_right_context is None:
        previous_right_context = [
            np.frombuffer(data_queue.get(), dtype=np.float32) for _ in range(1)
        ]

    # Get 4 segments.
    segments = [
        np.frombuffer(data_queue.get(), dtype=np.float32) for _ in range(4)
    ]

    current_input = previous_right_context + segments

    with torch.no_grad():
        print("hitting transcription branch")
        transcribe(np.concatenate(current_input), should_print=should_print)

    # Save right context.
    previous_right_context = current_input[-1:]

In [ ]:
wrapper = get_demo_wrapper()
print("Loaded model.")

In [ ]:
data_queue = queue.Queue()

In [ ]:
state = None
hypo = None

In [ ]:
previous_right_context = None

In [ ]:
# Emformer is configured with input segment size of 4 and right context size of 1.
# Pre- time reduction with factor 4, then, we have an input segment size of 16 and
# right context size of 4 going into RNN-T.
# With a hop length of 160 samples, we then have 16 * 160 = 2560 samples in the input segment
# and 4 * 160 = 640 samples in the right context.
# Then, since the lowest common factor between 640 and 3600 is 640, we'll
# read from the stream in 640-sample increments.

In [ ]:
path_seven = "sc_v01_test/seven/0ea0e2f4_nohash_0.wav"

In [ ]:
with wave.open(path_seven) as w:
    framerate = w.getframerate()
    frames = w.getnframes()
    channels = w.getnchannels()
    width = w.getsampwidth()
    print('sampling rate:', framerate, 'Hz')
    print('length:', frames, 'samples')
    print('channels:', channels)
    print('sample width:', width, 'bytes')

In [ ]:
p = pyaudio.PyAudio()

In [1]:
# We need to initialize the model by evaluating
# a few samples.
# If we skip this, evaluation latency will become
# prohibitively large.

print("Initializing model...")
for _ in range(10):
    print(_)
    process(should_print=False)

print("Initialization complete.")

In [ ]:
data_queue = queue.Queue()
previous_right_context = None
state = None
prev_hypo = None

In [ ]:
wf = wave.open(path_seven, 'rb')

In [ ]:
stream = p.open(
    format=pyaudio.paFloat32,
    channels=channels,
    rate=framerate,
    input=False,
    output=True,
    frames_per_buffer=640,
    stream_callback=callback,
)

In [ ]:
stream.start_stream()

In [ ]:
while stream.is_active():
    print("START")
    process(should_print=True)
    print("END")

In [ ]:
stream.stop_stream()
stream.close()

In [ ]:
wf.close()

In [ ]:
p.terminate()

# SCRATCH

In [3]:
for i in range(0, 2560, 640):
    print(i, i+640)

0 640
640 1280
1280 1920
1920 2560


In [4]:
from torchaudio.io import StreamReader

RuntimeError: torchaudio is not compiled with FFmpeg integration. Please set USE_FFMPEG=1 when compiling torchaudio.